AudioCraft github: https://github.com/facebookresearch/audiocraft  
AudioCraft API documentation: https://facebookresearch.github.io/audiocraft/api_docs/audiocraft/index.html  


AudioCraft - MusicGen: https://github.com/facebookresearch/audiocraft/blob/main/docs/MUSICGEN.md  

In [ ]:
# Deal with the bug ''NotImplementedError: A UTF-8 locale is required. Got ANSI_X3.4-1968 '
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!pip show torch

Name: torch
Version: 2.0.1+cu118
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org/
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD-3
Location: /usr/local/lib/python3.10/dist-packages
Requires: filelock, jinja2, networkx, sympy, triton, typing-extensions
Required-by: fastai, torchaudio, torchdata, torchtext, torchvision, triton


In [1]:
base_path='/content/drive/MyDrive/Wire_Note_Project/try_audio_to_sheet'

In [2]:
%cd f{base_path}
%pwd

[Errno 2] No such file or directory: 'f/content/drive/MyDrive/Wire_Note_Project/try_audio_to_sheet'
/content


'/content'

In [ ]:
!pip install -U audiocraft

In [7]:
import os
import torchaudio
from audiocraft.models import MusicGen
from audiocraft.data.audio import audio_write

In [9]:
output_path=os.path.join(base_path,'output')
os.makedirs(output_path, exist_ok=True)
os.path.exists(output_path)

True

Generate melody from video

In [ ]:
video_path=os.path.join(base_path,'test_audio.mp3')
os.path.exists(video_path)

True

In [ ]:
model = MusicGen.get_pretrained('facebook/musicgen-small')

print(model)

In [ ]:
melody_model = MusicGen.get_pretrained('facebook/musicgen-melody')

print(melody_model)

Downloading: "https://dl.fbaipublicfiles.com/demucs/hybrid_transformer/955717e8-8726e21a.th" to /root/.cache/torch/hub/checkpoints/955717e8-8726e21a.th
100%|██████████| 80.2M/80.2M [00:00<00:00, 129MB/s]


In [ ]:
model.set_generation_params(duration=8)  # generate 8 seconds.
wav = model.generate_unconditional(4)    # generates 4 unconditional audio samples
descriptions = ['happy rock', 'energetic EDM', 'sad jazz']
wav = model.generate(descriptions)  # generates 3 samples.

melody, sr = torchaudio.load(video_path)
# generates using the melody from the given audio and the provided descriptions.
wav = melody_model.generate_with_chroma(descriptions, melody[None].expand(3, -1, -1), sr)

for idx, one_wav in enumerate(wav):
    # Will save under {idx}.wav, with loudness normalization at -14 db LUFS.
    audio_write(f'{idx}', one_wav.cpu(), model.sample_rate, strategy="loudness", loudness_compressor=True)



Text to video

In [3]:
!pip install git+https://github.com/huggingface/transformers.git

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-73rqnx0b
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-73rqnx0b
  Resolved https://github.com/huggingface/transformers.git to commit c3ecf2d95d6a9f614d968af2f8b4e317f381e5ec
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 45.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.3 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.34.0.dev0-py3-none-any.whl size=7701221 sha256=030c5eb995ac576080cafc8001b80385dc4dfbb811a4f0c6e084a3929fbbf2c8
  Stored in directory: /tmp/pip-ephem-wheel-c

In [4]:
from transformers import AutoProcessor, MusicgenForConditionalGeneration

In [5]:
processor = AutoProcessor.from_pretrained("facebook/musicgen-small")
model = MusicgenForConditionalGeneration.from_pretrained("facebook/musicgen-small")

text=['happy weekend,alive,piano']
inputs = processor(
    text=text,
    padding=True,
    return_tensors="pt",
)

audio_values = model.generate(**inputs, max_new_tokens=256)

Loading the tokenizer from the `special_tokens_map.json` and the `added_tokens.json` will be removed in `transformers 5`,  it is kept for forward compatibility, but it is recommended to update your `tokenizer_config.json` by uploading it again. You will see the new `added_tokens_decoder` attribute that will store the relevant information.


In [10]:
import scipy

output_path_1=os.path.join(output_path,"musicgen_out-3.wav")

sampling_rate = model.config.audio_encoder.sampling_rate
scipy.io.wavfile.write(output_path_1, rate=sampling_rate, data=audio_values[0, 0].numpy())

In [11]:
from google.colab import files

files.download(output_path_1)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>